In [ ]:
!pip install --quiet mrjob==0.7.4

     |████████████████████████████████| 439 kB 4.9 MB/s 


In [ ]:
from google.colab import drive  #connecting google drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%%file q1.py
from mrjob.job import MRJob
import numpy as np
import heapq
from heapq import heapify, heappush, heappop
import re

Test_set = "/content/gdrive/MyDrive/datasets/Datasets/test_set_med.csv"  #for loading in mapper init before every mapper call

class KNNJob(MRJob):
    def mapper_init(self):   #is called before every mapper runs
      f = open(Test_set, 'r')  #opening the test dataset
      self.var=1
      self.k=31   # 'K' is declred here which can be used in mapper below
      lines = f.readlines()
      my_list = []  #creating empty list
      m=1
      for line in lines:  #this loop is just traversing test dataset and copying all lines in "my_list" except 1st row(Label Row)
        if (m!=1):
          record = line.split(',')  #splitting line of record
          my_list.append((int(record[0]), int(record[1]), int(record[2]), int(record[3]), int(record[4]), int(record[5]), 
                          int(record[6]), int(record[7]), int(record[8]), int(record[9]), int(record[10])))
          
        m=m+1
      self.my_list = my_list   #making this list such that in can be accessed in mapper 
      
    def mapper(self, _, line):
      record= re.split(',', line)   #splitting line of record
      if (record[10]!="hand"):   #for not reading 1st row (Label Row)
        point1 = np.array((int(record[0]), int(record[1]), int(record[2]), int(record[3]), int(record[4]), int(record[5]), 
                          int(record[6]), int(record[7]), int(record[8]), int(record[9])))   #Features of a particular Training Row

        for i in self.my_list:  #Traversing whole Test Set for above particular Training Row
          point2=np.array((int(i[1]), int(i[2]), int(i[3]), int(i[4]), int(i[5]),    #Features of current Test row
                          int(i[6]), int(i[7]), int(i[8]), int(i[9]), int(i[10])))
          x=np.linalg.norm(point1 - point2)   #Euclidean Distance between Test Row & Training ROw
          yield int(i[0]), [x, int(record[10])]   # key, value pair as ID_of_Test_Row -> {Euclidean Distance, Class of corresponding Training Dataset}
      

    def reducer_init(self):  #is called before every reducer call
      max_heap = [] 
      heapq._heapify_max(max_heap)   #creating max_heap
      self.heap=max_heap
      self.k=31
      arr1= []    #creating array of size 10 and initilizing it with value 0
      for a in range(0,10):
        arr1.append(0)

      self.arr1=arr1

    def reducer(self, rec, my_list):
      for a in my_list:   #traversing the list of key=rec having elements as {distance , class}
        heappush(self.heap, [a[0], a[1]])  #putting it in max_heap
        if(len(self.heap) > self.k):   #if size of heap increases "K" then we pop out max element as it is a max heap
          heapq._heappop_max(self.heap)

      while (len(self.heap) > 0):   #we have our nearest "K" elements in heap
        element=heappop(self.heap)
        self.arr1[element[1]]+=1    #storing count of class in the array defined in reducer init

      ele= -1
      class1= -1
      for a in range(0,10):   #this loop tells us which class has maximum count
        if (self.arr1[a]>=ele):
          ele=self.arr1[a]
          class1=a

      yield rec, class1   #yield "ID_of_Test_Set, Class_in_which_it_is_classified"


if __name__ == '__main__':
    KNNJob.run()


Writing q1.py


In [ ]:
!python q1.py "/content/gdrive/MyDrive/datasets/Datasets/traindata_medium.csv"  

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/q1.root.20221218.032110.779124
Running step 1 of 1...
job output is in /tmp/q1.root.20221218.032110.779124/output
Streaming final output from /tmp/q1.root.20221218.032110.779124/output...
1441	0
1442	0
1443	0
1444	0
1445	0
1446	0
1447	0
1448	0
1449	0
145	0
1450	0
1451	0
1452	0
1453	0
1454	0
1455	0
1456	0
1457	0
1458	0
1459	0
146	0
1460	0
1461	0
1462	0
1463	0
1464	0
1465	0
1466	0
1467	0
1468	0
1469	0
147	0
1470	0
1471	0
1472	0
1473	0
1474	0
1475	0
1476	0
1477	0
1478	0
1479	0
148	0
1480	0
1481	0
1482	0
1483	0
1484	0
1485	0
1486	0
1487	0
1488	0
1489	0
149	0
1490	0
1491	0
1492	0
1493	0
1494	0
1495	0
1496	0
1497	0
1498	0
1499	0
15	0
150	0
1500	0
1501	0
1502	0
1503	0
1504	0
1505	0
1506	0
1507	0
1508	0
1509	0
151	0
1510	0
1511	0
1512	0
1513	0
1514	0
1515	0
1516	0
1517	0
1518	0
1519	0
152	0
1520	0
1521	0
1522	0
1523	0
1524	0
1525	0
1526	0
1527	0
1528	0
1529	0
153	0
1530	0
15